<a href="https://colab.research.google.com/github/vinit-narula/AI-ML/blob/main/3_MNIST_Udacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classifying images of Clothing


In [ ]:
!pip install -U tensorflow_datasets

     |████████████████████████████████| 3.7MB 12.7MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
